## Data Cleaning

In [34]:
import pandas as pd
import numpy as np
import os
import pickle
import boto3
s3 = boto3.resource('s3')
bucket_name = "msds-practicum-carey"

import re
import spacy
#nlp = spacy.load("en_core_web_sm")

nlp = spacy.load('en_core_web_sm', disable=['ner', 'parser'])
#nlp.add_pipe(nlp.create_pipe('sentencizer'))

import nltk
from nltk import FreqDist
import string

from tqdm import tqdm_notebook as tqdm
tqdm().pandas()

/Users/scarey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:21: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


/Users/scarey/anaconda3/lib/python3.7/site-packages/tqdm/std.py:654: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [35]:
with open('outdata/tweet_df.pkl', 'wb') as data:
    s3.Bucket(bucket_name).download_fileobj('tweet_df.pkl', data)
    
with open('outdata/tweet_df.pkl', 'rb') as data:
    tweet_df = pickle.load(data)
    
os.remove('outdata/tweet_df.pkl')

In [36]:
tweet_df

,tweet,class
0,RT @aafb: Congrats to ⁦@RepOHalleran⁩ &amp; ⁦@...,L
1,Great to meet the new Lake County Farm Bureau ...,L
2,Congratulations to @waynestcollege women's rug...,C
3,Great to meet with the Erickson Air Crane team...,C
4,Always wonderful to be part of the Back to Sch...,L
...,...,...
1350301,We should be upholding the National Environmen...,L
1350302,"If anything is to be investigated, I think we ...",C
1350303,TODAY: Federal judge rules in favor of House R...,C
1350304,"In the words of an old proverb, ""A hit dog wil...",L


In [37]:
# import stopwords 
stopwords = nltk.corpus.stopwords.words('english') 
stopwords.extend(['RT'])

In [38]:
def remove_punct(text):
    
    regex = r'[^\w\s]'
    
    return re.sub(regex,'',text)

def tokenize(text):
    
    tokens = nltk.word_tokenize(text)
    
    return tokens

def remove_stopwords(words):
  
    
    filtered = filter(lambda word: word not in stopwords, words)
    
    return list(filtered)

def lemmatize(text, nlp=nlp):
    
    doc = nlp(text)
    
    lemmatized = [token.lemma_ for token in doc]
    
    return " ".join(lemmatized)

def clean_text(df):
   
    df["clean_tweets"] = [lemmatize(x) for x in tqdm(df['clean_tweets'].tolist())]
    print('done')
    return df

# Gets rid of emojis and some oddly formated strings
def remove_emoji(inputString):
    return inputString.encode('ascii', 'ignore').decode('ascii')

In [39]:
tweet_df['clean_tweets'] = tweet_df['tweet'].progress_apply(lambda x: re.sub('http://\S+', '', x))
tweet_df['clean_tweets'] = tweet_df['clean_tweets'].progress_apply(lambda x: re.sub('https://\S+', '', x))
tweet_df['clean_tweets'] = tweet_df['clean_tweets'].progress_apply(lambda x: re.sub('@\S+', '', x))
tweet_df['clean_tweets'] = tweet_df['clean_tweets'].progress_apply(lambda x: remove_emoji(x))
tweet_df['clean_tweets'] = tweet_df['clean_tweets'].progress_apply(lambda x: re.sub('\n', '', x))
tweet_df['clean_tweets'] = tweet_df['clean_tweets'].progress_apply(lambda x: re.sub('&amp;', '', x))
tweet_df['clean_tweets'] = tweet_df['clean_tweets'].progress_apply(lambda x: re.sub('&amp', '', x))
tweet_df['clean_tweets'] = tweet_df['clean_tweets'].progress_apply(lambda x: re.sub(r'[^\w\s]', '', x))
tweet_df['clean_tweets'] = tweet_df['clean_tweets'].progress_apply(lambda x: tokenize(x))
tweet_df['clean_tweets'] = tweet_df['clean_tweets'].progress_apply(lambda x : remove_stopwords(x))
tweet_df['clean_tweets'] = tweet_df['clean_tweets'].progress_apply(lambda x : " ".join(x) )

In [40]:
import multiprocessing
parts = multiprocessing.cpu_count()
parts

12

In [41]:
import dask.dataframe as ddf
from dask.diagnostics import ProgressBar

dask_df = ddf.from_pandas(tweet_df, npartitions=parts)
result = dask_df.map_partitions(clean_text, meta=tweet_df)
with ProgressBar():
    df = result.compute(scheduler='processes')

[                                        ] | 0% Completed |  2.6s

/Users/scarey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


[                                        ] | 0% Completed |  3.6s

[                                        ] | 0% Completed |  7.7s

/Users/scarey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


...

[                                        ] | 0% Completed | 44.2s

[                                        ] | 0% Completed | 49.8s

/Users/scarey/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:30: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


[                                        ] | 0% Completed | 49.9s

[                                        ] | 0% Completed | 19min 28.8s
done
[###                                     ] | 8% Completed | 19min 41.2s
done
[######                                  ] | 16% Completed | 19min 49.5s
done
[##########                              ] | 25% Completed | 19min 55.6s
[##########                              ] | 25% Completed | 19min 55.7sdone
[#############                           ] | 33% Completed | 20min  4.5s
done
[################                        ] | 41% Completed | 20min 10.3s
done
[################                        ] | 41% Completed | 20min 11.7s
done
[#######################                 ] | 58% Completed | 20min 14.0s
[#######################                 ] | 58% Completed | 20min 14.1sdone
[##########################              ] | 66% Completed | 20min 16.6s
done
[##############################          ] | 75% Completed | 20min 18.8s
done
[##############################          ] | 75% Completed | 20min 20.4s
done


In [42]:
df


,tweet,class,clean_tweets
0,RT @aafb: Congrats to ⁦@RepOHalleran⁩ &amp; ⁦@...,L,congrat appointment look forward work together h
1,Great to meet the new Lake County Farm Bureau ...,L,great meet new lake county farm bureau executi...
2,Congratulations to @waynestcollege women's rug...,C,congratulation women rugby win sixth national ...
3,Great to meet with the Erickson Air Crane team...,C,great meet erickson air crane team medford tod...
4,Always wonderful to be part of the Back to Sch...,L,always wonderful part back school jam resource...
...,...,...,...
1350301,We should be upholding the National Environmen...,L,-PRON- uphold national environmental policy ac...
1350302,"If anything is to be investigated, I think we ...",C,if anything investigate -PRON- think need inve...
1350303,TODAY: Federal judge rules in favor of House R...,C,today federal judge rule favor house republica...
1350304,"In the words of an old proverb, ""A hit dog wil...",L,in word old proverb a hit dog holler


In [ ]:
with open('outdata/tweets_clean_df.pkl', 'wb') as f:
    pickle.dump(df, f)
    
s3.meta.client.upload_file('outdata/tweets_clean_df.pkl',
                           bucket_name,
                           'tweets_clean_df.pkl')


In [ ]:
os.remove('outdata/tweets_clean_df.pkl')